<a href="https://colab.research.google.com/github/sunyingjian/AI-in-well-logging/blob/master/%3CConcise%20code%3ESVM%20Random%20Forests%20GBDT%20XGBoost%20facies%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 导入数据

In [ ]:
!git clone https://github.com/sunyingjian/tutorials-2016.git

fatal: destination path 'tutorials-2016' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/sunyingjian/numpy-.git

Cloning into 'numpy-'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 418 (delta 4), reused 0 (delta 0), pack-reused 403
Receiving objects: 100% (418/418), 202.27 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (118/118), done.
Checking out files: 100% (206/206), done.


In [ ]:
! /opt/bin/nvidia-smi

Thu Feb 25 06:48:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
%matplotlib inline

In [ ]:
training_data = pd.read_csv('/content/numpy-/美国分井/train_data_处理完成 (1).csv')
blind_data = pd.read_csv('/content/numpy-/美国分井/blind_data_处理完成.csv')

In [ ]:
blind = pd.read_csv('/content/numpy-/美国油田/blind_data_wellname.csv')

In [ ]:
training_data

,Unnamed: 0,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies,Well Name
0,0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0.557385,0.0,1.000000,3,7
1,1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0.494046,0.0,0.978788,3,7
2,2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0.430707,0.0,0.956566,3,7
3,3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0.418039,0.0,0.935354,3,7
4,4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0.405371,0.0,0.914141,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...
3227,3227,0.923077,0.996357,0.155858,0.646070,0.565833,0.129373,0.432860,1.0,0.681818,5,0
3228,3228,0.923077,0.997268,0.145818,0.650055,0.576589,0.144021,0.398277,1.0,0.673737,5,0
3229,3229,0.923077,0.998179,0.169829,0.657359,0.594401,0.144021,0.378769,1.0,0.665657,5,0
3230,3230,0.923077,0.999089,0.177978,0.658023,0.598516,0.138135,0.373955,1.0,0.657576,5,0


In [ ]:
training_data1 = pd.read_csv('/content/numpy-/block_TCN_处理完成.csv')

In [ ]:
training_data1

,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies,Well Name
0,0.076923,0.000000,0.575189,0.289898,0.464680,0.348468,0.385598,0,1.000000,3,8
1,0.076923,0.001996,0.604475,0.272564,0.647994,0.433656,0.373814,0,0.973764,3,8
2,0.076923,0.003992,0.617873,0.270173,0.690625,0.473574,0.347765,0,0.948537,3,8
3,0.076923,0.005988,0.623511,0.270173,0.669310,0.488738,0.344505,0,0.922301,3,8
4,0.076923,0.007984,0.612382,0.265989,0.618152,0.473351,0.343765,0,0.896065,3,8
...,...,...,...,...,...,...,...,...,...,...,...
900,0.923077,0.992016,0.555516,0.335326,0.645820,0.398800,0.380910,1,0.399596,4,9
901,0.923077,0.994012,0.665442,0.361626,0.612994,0.411066,0.395389,1,0.387487,6,9
902,0.923077,0.996008,0.343731,0.371189,0.528542,0.474867,0.408255,1,0.375378,6,9
903,0.923077,0.998004,0.980854,0.325762,0.570746,0.474867,0.402142,1,0.363269,6,9


In [ ]:
training_data = training_data.drop(columns='Unnamed: 0')

In [ ]:
training_data = pd.concat([training_data,training_data1])

In [ ]:
training_data

,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies,Well Name
0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0.557385,0.0,1.000000,3,7
1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0.494046,0.0,0.978788,3,7
2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0.430707,0.0,0.956566,3,7
3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0.418039,0.0,0.935354,3,7
4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0.405371,0.0,0.914141,3,7
...,...,...,...,...,...,...,...,...,...,...,...
900,0.923077,0.992016,0.555516,0.335326,0.645820,0.398800,0.380910,1.0,0.399596,4,9
901,0.923077,0.994012,0.665442,0.361626,0.612994,0.411066,0.395389,1.0,0.387487,6,9
902,0.923077,0.996008,0.343731,0.371189,0.528542,0.474867,0.408255,1.0,0.375378,6,9
903,0.923077,0.998004,0.980854,0.325762,0.570746,0.474867,0.402142,1.0,0.363269,6,9


In [ ]:
training_data = training_data.reset_index()

In [ ]:
training_data

,index,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies,Well Name
0,0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0.557385,0.0,1.000000,3,7
1,1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0.494046,0.0,0.978788,3,7
2,2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0.430707,0.0,0.956566,3,7
3,3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0.418039,0.0,0.935354,3,7
4,4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0.405371,0.0,0.914141,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...
4132,900,0.923077,0.992016,0.555516,0.335326,0.645820,0.398800,0.380910,1.0,0.399596,4,9
4133,901,0.923077,0.994012,0.665442,0.361626,0.612994,0.411066,0.395389,1.0,0.387487,6,9
4134,902,0.923077,0.996008,0.343731,0.371189,0.528542,0.474867,0.408255,1.0,0.375378,6,9
4135,903,0.923077,0.998004,0.980854,0.325762,0.570746,0.474867,0.402142,1.0,0.363269,6,9


In [ ]:
training_data = training_data.drop(columns='index')

In [ ]:
train_data = training_data

In [ ]:
training_data

,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies,Well Name
0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0.557385,0.0,1.000000,3,7
1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0.494046,0.0,0.978788,3,7
2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0.430707,0.0,0.956566,3,7
3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0.418039,0.0,0.935354,3,7
4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0.405371,0.0,0.914141,3,7
...,...,...,...,...,...,...,...,...,...,...,...
4132,0.923077,0.992016,0.555516,0.335326,0.645820,0.398800,0.380910,1.0,0.399596,4,9
4133,0.923077,0.994012,0.665442,0.361626,0.612994,0.411066,0.395389,1.0,0.387487,6,9
4134,0.923077,0.996008,0.343731,0.371189,0.528542,0.474867,0.408255,1.0,0.375378,6,9
4135,0.923077,0.998004,0.980854,0.325762,0.570746,0.474867,0.402142,1.0,0.363269,6,9


In [ ]:
blind_data = blind_data.drop(columns=['Unnamed: 0','Well Name'])

In [ ]:
train_data = train_data.drop(columns=['Well Name'])

In [21]:
train_data

,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies
0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0.557385,0.0,1.000000,3
1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0.494046,0.0,0.978788,3
2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0.430707,0.0,0.956566,3
3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0.418039,0.0,0.935354,3
4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0.405371,0.0,0.914141,3
...,...,...,...,...,...,...,...,...,...,...
4132,0.923077,0.992016,0.555516,0.335326,0.645820,0.398800,0.380910,1.0,0.399596,4
4133,0.923077,0.994012,0.665442,0.361626,0.612994,0.411066,0.395389,1.0,0.387487,6
4134,0.923077,0.996008,0.343731,0.371189,0.528542,0.474867,0.408255,1.0,0.375378,6
4135,0.923077,0.998004,0.980854,0.325762,0.570746,0.474867,0.402142,1.0,0.363269,6


In [22]:
train_x = train_data.drop(columns='Facies')

In [23]:
train_y = train_data['Facies'].values

In [24]:
blind_x = blind_data.drop(columns='Facies')
blind_y = blind_data['Facies'].values

# SVM

In [47]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [48]:
from sklearn import svm

In [49]:
clf = svm.SVC()

In [50]:
import sklearn
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [51]:
clf.fit(train_x, train_y)
y_pred= clf.predict(blind_x)
print("Accuracy : %.4f" % sklearn.metrics.accuracy_score(blind_y, y_pred))
print(f'混淆矩阵')
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, y_pred))
print('classification report')
print(metrics.classification_report(blind_y, y_pred))
print(precision_score(blind_y, y_pred, average='weighted'))
print(recall_score(blind_y, y_pred, average='weighted'))
print(f1_score(blind_y, y_pred,average='weighted'))

Accuracy : 0.5277
混淆矩阵
confusion matrix
[[  0  20   0   0   0   0   0   0   0]
 [  0 118  37   0   0   0   0   4   0]
 [  0  35  52  23   0   1  26  16   1]
 [  0   0   0  29   1  35   0   0   0]
 [  0   0   2   2   0  37   0   8   0]
 [  0   0   0  15   2  58   0  49   3]
 [  0   0   0   2   1   6  11   6   0]
 [  0   0   4   4   0  25   0 170   9]
 [  0   0   0   0   0   0   0  18   0]]
classification report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        20
           2       0.68      0.74      0.71       159
           3       0.55      0.34      0.42       154
           4       0.39      0.45      0.41        65
           5       0.00      0.00      0.00        49
           6       0.36      0.46      0.40       127
           7       0.30      0.42      0.35        26
           8       0.63      0.80      0.70       212
           9       0.00      0.00      0.00        18

    accuracy                           0.53 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('svm.csv')

# XGBoost

In [54]:
from sklearn import metrics
import xgboost
import numpy as np
# 构建XGBoost分类器
xgboost = xgboost.XGBClassifier()
# 使用重抽样后的数据，对其建模
xgboost.fit(train_x,train_y)
# 将模型运用到测试数据集中
y_pred = xgboost.predict(blind_x) #传入的是array
print("Accuracy : %.4f" % sklearn.metrics.accuracy_score(blind_y, y_pred))
print(f'混淆矩阵')
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, y_pred))
print('classification report')
print(metrics.classification_report(blind_y, y_pred))
print(precision_score(blind_y, y_pred, average='weighted'))
print(recall_score(blind_y, y_pred, average='weighted'))
print(f1_score(blind_y, y_pred,average='weighted'))

Accuracy : 0.4964
混淆矩阵
confusion matrix
[[  2  10   8   0   0   0   0   0   0]
 [ 44  49  62   0   0   1   0   3   0]
 [  0  17  76  15   0   4  34   8   0]
 [  0   0   3  37  11  13   0   1   0]
 [  0   0   3   2   8  24   0  12   0]
 [  0   0  16   6   3  64   0  37   1]
 [  0   0   0   2   5   1  11   7   0]
 [  0   1   5   4   0  32   4 165   1]
 [  0   0   0   0   0   0   0  18   0]]
classification report
              precision    recall  f1-score   support

           1       0.04      0.10      0.06        20
           2       0.64      0.31      0.42       159
           3       0.44      0.49      0.46       154
           4       0.56      0.57      0.56        65
           5       0.30      0.16      0.21        49
           6       0.46      0.50      0.48       127
           7       0.22      0.42      0.29        26
           8       0.66      0.78      0.71       212
           9       0.00      0.00      0.00        18

    accuracy                           0.50 

In [55]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('XGBoost.csv')

# GBDT

In [57]:
max_depth=40
max_features=100
max_features=100
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.metrics
gbm0 = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, random_state=42).fit(train_x, train_y)
y_pred= gbm0.predict(blind_x)
print("Accuracy : %.4f" % sklearn.metrics.accuracy_score(blind_y, y_pred))
print(f'混淆矩阵')
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, y_pred))
print('classification report')
print(metrics.classification_report(blind_y, y_pred))
print(precision_score(blind_y, y_pred, average='weighted'))
print(recall_score(blind_y, y_pred, average='weighted'))
print(f1_score(blind_y, y_pred,average='weighted'))

Accuracy : 0.4361
混淆矩阵
confusion matrix
[[  4   9   7   0   0   0   0   0   0]
 [ 40  36  79   0   0   0   0   4   0]
 [  1  13  87   9   0   1  34   6   3]
 [  0   0   9  17  22  11   0   6   0]
 [  0   0   4   2   1  29   1  11   1]
 [  0   0  14   5   2  60   0  39   7]
 [  0   0   2   1   5   7   3   7   1]
 [  0   1   5   4   1  26   9 152  14]
 [  0   0   0   0   0   0   0  16   2]]
classification report
              precision    recall  f1-score   support

           1       0.09      0.20      0.12        20
           2       0.61      0.23      0.33       159
           3       0.42      0.56      0.48       154
           4       0.45      0.26      0.33        65
           5       0.03      0.02      0.03        49
           6       0.45      0.47      0.46       127
           7       0.06      0.12      0.08        26
           8       0.63      0.72      0.67       212
           9       0.07      0.11      0.09        18

    accuracy                           0.44 

In [58]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('GBDT.csv')

# 随机森林

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(train_x,train_y)
print('model')
print(rfmodel) 

ypredrf1 = rfmodel.predict(blind_x)
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, ypredrf1))
print('classification report')
print(metrics.classification_report(blind_y, ypredrf1))
print('Accuracy : %f' % (metrics.accuracy_score(blind_y, ypredrf1)))
print(precision_score(blind_y, ypredrf1, average='weighted'))
print(recall_score(blind_y, ypredrf1, average='weighted'))
print(f1_score(blind_y, ypredrf1,average='weighted'))

model
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
confusion matrix
[[  0  15   5   0   0   0   0   0   0]
 [ 10  68  76   0   1   2   0   2   0]
 [  0  20  69  11   3  13  24  14   0]
 [  0   0   0  19  21  11   0  14   0]
 [  0   1   2   0   0  17   0  29   0]
 [  0   0  13   5   2  49   0  58   0]
 [  0   0   0   1   3   4   2  16   0]
 [  0   2   2   2   6  24   0 169   7]
 [  0   0   0   0   0   0   0  18   0]]
classification report
              precision    recall  f1-score   support

 

In [60]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('随机森林.csv')

# SMOTE 系列方法

## SMOTE过采样

In [61]:
a = training_data['Facies'].value_counts()

In [62]:
a

2    940
3    780
8    686
6    582
5    296
4    271
1    268
9    173
7    141
Name: Facies, dtype: int64

In [63]:
from imblearn.over_sampling import SMOTE
from collections import Counter
sm = SMOTE(sampling_strategy={5:500, 4:500, 1:500, 9:500, 7:500},random_state=24)
train_x_sm, train_y_sm = sm.fit_sample(train_x, train_y)
train_x_sm = pd.DataFrame(train_x_sm)
train_y_sm = pd.DataFrame(train_y_sm)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [64]:
train_x_sm

,0,1,2,3,4,5,6,7,8
0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0.557385,0.0,1.000000
1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0.494046,0.0,0.978788
2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0.430707,0.0,0.956566
3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0.418039,0.0,0.935354
4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0.405371,0.0,0.914141
...,...,...,...,...,...,...,...,...,...
5483,0.769231,0.896945,0.131065,0.174967,0.566944,0.326708,0.706382,1.0,0.598604
5484,0.769231,0.899811,0.118115,0.130846,0.585812,0.295335,0.836811,1.0,0.545584
5485,0.769231,0.904665,0.114625,0.087619,0.570939,0.357633,0.665547,1.0,0.456075
5486,0.307692,0.800601,0.088778,0.560680,0.481497,0.176231,0.603506,1.0,0.593949


In [65]:
train_x_sm.columns = ['Formation', 'Depth', 'GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']

In [66]:
train_x_sm

,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0.557385,0.0,1.000000
1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0.494046,0.0,0.978788
2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0.430707,0.0,0.956566
3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0.418039,0.0,0.935354
4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0.405371,0.0,0.914141
...,...,...,...,...,...,...,...,...,...
5483,0.769231,0.896945,0.131065,0.174967,0.566944,0.326708,0.706382,1.0,0.598604
5484,0.769231,0.899811,0.118115,0.130846,0.585812,0.295335,0.836811,1.0,0.545584
5485,0.769231,0.904665,0.114625,0.087619,0.570939,0.357633,0.665547,1.0,0.456075
5486,0.307692,0.800601,0.088778,0.560680,0.481497,0.176231,0.603506,1.0,0.593949


## SMOTE SVM

In [42]:
clf = svm.SVC()

In [43]:
clf.fit(train_x_sm, train_y_sm)
y_pred= clf.predict(blind_x)
print("Accuracy : %.4f" % sklearn.metrics.accuracy_score(blind_y, y_pred))
print(f'混淆矩阵')
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, y_pred))
print('classification report')
print(metrics.classification_report(blind_y, y_pred))
print(precision_score(blind_y, y_pred, average='weighted'))
print(recall_score(blind_y, y_pred, average='weighted'))
print(f1_score(blind_y, y_pred,average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy : 0.4699
混淆矩阵
confusion matrix
[[  9  11   0   0   0   0   0   0   0]
 [ 36  82  37   0   0   0   0   4   0]
 [  3  32  52  21   4   1  26   6   9]
 [  0   0   0  38   5  22   0   0   0]
 [  0   0   2   2   2  36   0   6   1]
 [  0   0   0  17   4  52   1  42  11]
 [  0   0   0   3   3   3  16   1   0]
 [  0   0   4   7   2  22   4 138  35]
 [  0   0   0   0   0   0   0  17   1]]
classification report
              precision    recall  f1-score   support

           1       0.19      0.45      0.26        20
           2       0.66      0.52      0.58       159
           3       0.55      0.34      0.42       154
           4       0.43      0.58      0.50        65
           5       0.10      0.04      0.06        49
           6       0.38      0.41      0.40       127
           7       0.34      0.62      0.44        26
           8       0.64      0.65      0.65       212
           9       0.02      0.06      0.03        18

    accuracy                           0.47 

In [67]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('SMOTE_SVM.csv')

## SMOTE XGBoost

In [68]:
from sklearn import metrics
import xgboost
import numpy as np
# 构建XGBoost分类器
xgboost = xgboost.XGBClassifier()
# 使用重抽样后的数据，对其建模
xgboost.fit(train_x_sm,train_y_sm)
# 将模型运用到测试数据集中
y_pred = xgboost.predict(blind_x) #传入的是array
print("Accuracy : %.4f" % sklearn.metrics.accuracy_score(blind_y, y_pred))
print(f'混淆矩阵')
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, y_pred))
print('classification report')
print(metrics.classification_report(blind_y, y_pred))
print(precision_score(blind_y, y_pred, average='weighted'))
print(recall_score(blind_y, y_pred, average='weighted'))
print(f1_score(blind_y, y_pred,average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy : 0.4627
混淆矩阵
confusion matrix
[[  6   8   6   0   0   0   0   0   0]
 [ 59  28  68   0   0   3   0   1   0]
 [  3  18  72  12   0   1  35  12   1]
 [  0   0   3  40  15   7   0   0   0]
 [  0   0   5   3  10  23   2   6   0]
 [  0   0  17   6   3  61   3  35   2]
 [  0   0   1   2   3   2  16   2   0]
 [  0   0   5   7   3  29   9 150   9]
 [  0   0   0   0   0   0   0  17   1]]
classification report
              precision    recall  f1-score   support

           1       0.09      0.30      0.14        20
           2       0.52      0.18      0.26       159
           3       0.41      0.47      0.44       154
           4       0.57      0.62      0.59        65
           5       0.29      0.20      0.24        49
           6       0.48      0.48      0.48       127
           7       0.25      0.62      0.35        26
           8       0.67      0.71      0.69       212
           9       0.08      0.06      0.06        18

    accuracy                           0.46 

In [69]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('SMOTE_XGBoost.csv')

## SMOTE GBDT

In [70]:
max_depth=40
max_features=100
max_features=100
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.metrics
gbm0 = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, random_state=42).fit(train_x_sm, train_y_sm)
y_pred= gbm0.predict(blind_x)
print("Accuracy : %.4f" % sklearn.metrics.accuracy_score(blind_y, y_pred))
print(f'混淆矩阵')
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, y_pred))
print('classification report')
print(metrics.classification_report(blind_y, y_pred))
print(precision_score(blind_y, y_pred, average='weighted'))
print(recall_score(blind_y, y_pred, average='weighted'))
print(f1_score(blind_y, y_pred,average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy : 0.4578
混淆矩阵
confusion matrix
[[  4   9   7   0   0   0   0   0   0]
 [ 43  33  79   0   0   0   0   4   0]
 [  1  12  95   7   0   1  34   4   0]
 [  0   0   7  22  20  15   0   1   0]
 [  0   0   4   2   4  31   0   6   2]
 [  0   0  14   4   1  60   1  39   8]
 [  0   0   4   2   3   4  10   3   0]
 [  0   0   7   4   2  31  10 150   8]
 [  0   0   0   0   0   0   0  16   2]]
classification report
              precision    recall  f1-score   support

           1       0.08      0.20      0.12        20
           2       0.61      0.21      0.31       159
           3       0.44      0.62      0.51       154
           4       0.54      0.34      0.42        65
           5       0.13      0.08      0.10        49
           6       0.42      0.47      0.45       127
           7       0.18      0.38      0.25        26
           8       0.67      0.71      0.69       212
           9       0.10      0.11      0.11        18

    accuracy                           0.46 

In [71]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('SMOTE_GBDT.csv')

## SMOTE 随机森林

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(train_x_sm,train_y_sm)
print('model')
print(rfmodel) 

ypredrf1 = rfmodel.predict(blind_x)
print('confusion matrix')
print(metrics.confusion_matrix(blind_y, ypredrf1))
print('classification report')
print(metrics.classification_report(blind_y, ypredrf1))
print('Accuracy : %f' % (metrics.accuracy_score(blind_y, ypredrf1)))
print(precision_score(blind_y, ypredrf1, average='weighted'))
print(recall_score(blind_y, ypredrf1, average='weighted'))
print(f1_score(blind_y, ypredrf1,average='weighted'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


model
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
confusion matrix
[[  0  15   5   0   0   0   0   0   0]
 [  9  91  55   0   1   0   0   3   0]
 [  0  21  71  11   1   6  25  19   0]
 [  0   0   2  30  19   8   0   6   0]
 [  0   0   2   1   3  11   0  32   0]
 [  0   0  13   4   3  46   1  59   1]
 [  0   0   0   1   4   0   2  19   0]
 [  0   4   1   2   3  16   3 173  10]
 [  0   0   0   0   0   0   0  17   1]]
classification report
              precision    recall  f1-score   support

 

In [73]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('SMOTE_随机森林.csv')